# 19. RNN with Hamlet

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import nltk

import random
import numpy as np

## 19.1 Prepare Data

In [2]:
nltk.download("gutenberg")

[nltk_data] Downloading package gutenberg to /home/ubuntu/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [3]:
raw = nltk.corpus.gutenberg.raw("shakespeare-hamlet.txt")
print(raw[:500])

[The Tragedie of Hamlet by William Shakespeare 1599]


Actus Primus. Scoena Prima.

Enter Barnardo and Francisco two Centinels.

  Barnardo. Who's there?
  Fran. Nay answer me: Stand & vnfold
your selfe

   Bar. Long liue the King

   Fran. Barnardo?
  Bar. He

   Fran. You come most carefully vpon your houre

   Bar. 'Tis now strook twelue, get thee to bed Francisco

   Fran. For this releefe much thankes: 'Tis bitter cold,
And I am sicke at heart

   Barn. Haue you had quiet Guard?
  Fran. Not


## 19.2 Char to Dic

In [4]:
char2index = {}
index2char = []

In [5]:
for char in raw :
    if char not in char2index.keys() :
        char2index[char] = len(char2index)
        index2char.append(char)

In [6]:
char2index

{'[': 0,
 'T': 1,
 'h': 2,
 'e': 3,
 ' ': 4,
 'r': 5,
 'a': 6,
 'g': 7,
 'd': 8,
 'i': 9,
 'o': 10,
 'f': 11,
 'H': 12,
 'm': 13,
 'l': 14,
 't': 15,
 'b': 16,
 'y': 17,
 'W': 18,
 'S': 19,
 'k': 20,
 's': 21,
 'p': 22,
 '1': 23,
 '5': 24,
 '9': 25,
 ']': 26,
 '\n': 27,
 'A': 28,
 'c': 29,
 'u': 30,
 'P': 31,
 '.': 32,
 'n': 33,
 'E': 34,
 'B': 35,
 'F': 36,
 'w': 37,
 'C': 38,
 "'": 39,
 '?': 40,
 'N': 41,
 ':': 42,
 '&': 43,
 'v': 44,
 'L': 45,
 'K': 46,
 'Y': 47,
 ',': 48,
 'I': 49,
 'q': 50,
 'G': 51,
 'M': 52,
 'R': 53,
 '-': 54,
 'D': 55,
 'O': 56,
 'x': 57,
 ';': 58,
 'Q': 59,
 'z': 60,
 '(': 61,
 ')': 62,
 'V': 63,
 '!': 64,
 'j': 65,
 'Z': 66}

In [7]:
len(char2index)

67

In [8]:
char2vec = {}
eye = np.eye(len(char2index))

In [9]:
for item in char2index.keys() :
    char2vec[item] = eye[char2index[item],:]

In [10]:
char2vec['a']

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
data = np.array([char2vec[char] for char in raw])

## 19.3 Define Model

In [12]:
# Parameters:	
# input_size – The number of expected features in the input x
# hidden_size – The number of features in the hidden state h
# num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two RNNs together to form a stacked RNN, with the second RNN taking in outputs of the first RNN and computing the final results. Default: 1
# nonlinearity – The non-linearity to use. Can be either ‘tanh’ or ‘relu’. Default: ‘tanh’
# bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True
# batch_first – If True, then the input and output tensors are provided as (batch, seq, feature). Default: False
# dropout – If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
# bidirectional – If True, becomes a bidirectional RNN. Default: False

In [13]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
#         self.rnn = nn.GRU(input_size,hidden_size,num_layers)
        self.rnn = nn.RNN(input_size,hidden_size,num_layers, dropout=0.5)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        out,hidden = self.rnn(input.view(1,1,-1),hidden)
        out = self.fc(out.view(1,-1))
        return out,hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.num_layers, 1, self.hidden_size)).cuda()
        return hidden

In [14]:
model = RNN(len(data[0]), 500, len(data[0]), 1).cuda()

## 19.4 Train Model

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

In [16]:
step = 100
num_epochs = 5

In [17]:
for epoch in range(num_epochs):
    
    sp = list(range(0, len(data) - 2 * step, step))
    sp = np.add(sp, random.randint(0, step))
    random.shuffle(sp)
    
    for i in range(len(sp)) :
    
        hidden = model.init_hidden()

        cost = 0

        for pos in range(sp[i], sp[i] + step):
            X = Variable(torch.from_numpy(data[pos]).type(torch.FloatTensor)).cuda()
            y = torch.from_numpy(data[pos+1]).cuda()
            _, y = y.max(dim=0)

            pre, hidden = model(X,hidden)
            cost += loss(pre,Variable(y).cuda())

        cost.backward()
        
        nn.utils.clip_grad_norm(model.parameters(), 5)
        
        optimizer.step()

        if (i + 1) % 100 == 0 :
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
                     %(epoch+1, num_epochs, i + 1, len(sp), cost.data[0]))

Epoch [1/5], Iter [100/1627] Loss: 316.6672
Epoch [1/5], Iter [200/1627] Loss: 329.9896
Epoch [1/5], Iter [300/1627] Loss: 273.8569
Epoch [1/5], Iter [400/1627] Loss: 246.0619
Epoch [1/5], Iter [500/1627] Loss: 224.3812
Epoch [1/5], Iter [600/1627] Loss: 246.5113
Epoch [1/5], Iter [700/1627] Loss: 209.9599
Epoch [1/5], Iter [800/1627] Loss: 232.5223
Epoch [1/5], Iter [900/1627] Loss: 217.3411
Epoch [1/5], Iter [1000/1627] Loss: 212.8072
Epoch [1/5], Iter [1100/1627] Loss: 228.4046
Epoch [1/5], Iter [1200/1627] Loss: 249.0327
Epoch [1/5], Iter [1300/1627] Loss: 257.2166
Epoch [1/5], Iter [1400/1627] Loss: 235.2767
Epoch [1/5], Iter [1500/1627] Loss: 232.9294
Epoch [1/5], Iter [1600/1627] Loss: 224.7668
Epoch [2/5], Iter [100/1627] Loss: 205.2259
Epoch [2/5], Iter [200/1627] Loss: 224.1319
Epoch [2/5], Iter [300/1627] Loss: 208.9300
Epoch [2/5], Iter [400/1627] Loss: 147.5427
Epoch [2/5], Iter [500/1627] Loss: 237.7560
Epoch [2/5], Iter [600/1627] Loss: 217.4826
Epoch [2/5], Iter [700/16

## 19.5 Test Model

In [18]:
start_num = 1
text = index2char[start_num]

model.eval()
hidden = model.init_hidden()

X_test = Variable(torch.from_numpy(data[start_num]).type(torch.FloatTensor)).cuda()
    
for i in range(500) :

    pre, hidden = model(X_test, hidden)

    temp = pre.cpu().data.numpy()[0]

    best_5 = np.argsort(temp)[::-1][:5]
    
    temp = np.exp(temp[best_5])
    
    temp = temp / temp.sum()
    
    pre = np.random.choice(best_5, 1, p = temp)[0]
    
    curr_char = index2char[pre]
    
    text += curr_char
    
    X_test = Variable(torch.from_numpy(char2vec[curr_char]).type(torch.FloatTensor)).cuda()
    
print("* Generated Text : \n", text)

* Generated Text : 
 To to but serf a drage tide thy worde a Matenn

   Bam. The tow the thought that he my Lord, and with the mend is combetser,
Hadsinge,
And wothing tome of ich the bering

   Goor. I so farle: will ghe this to the Spory?
  Polin. Wist the bleay shall what all ay the within of ming our contor Kinge, If houe tid this wall tay teale.

  Ham. That it, warthis indore of my Sorle mine forme and be make of in mand the Merame,
Ile it thim this?
Thim it the Singoued inder words why camast and in that? With
